In [2]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
import pickle
from sklearn.model_selection import train_test_split
from tensorflow import keras
from IPython.display import display

from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
#from imutils import paths #might need to install imutils package

#import shutil
#from pycocotools.coco import COCO
#from pathlib import Path
#from PIL import Image

#import torch
#import torchvision
#from torch.utils.data import Dataset
#from torchvision.transforms import ToTensor
#from torchvision.io import read_image    #could not import because running older version of torchvision in learn env

/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


### Pulling data into Dataframes

In [3]:
# setting data paths and making list of each file type

path = '../Data/data/'

image_list = [x for x in os.listdir(path) if x.endswith('.jpeg')]

label_list = [x for x in os.listdir(path) if x.endswith('.txt')]

image_list.sort()
label_list.sort()

In [4]:
# checking image and label lists
image_list_check = [x[:-5] for x in image_list]
label_list_check = [x[:-4] for x in label_list]
                    
image_list_check == label_list_check

True

In [5]:
# creating initial dataframe from txt files (YOLO format)
df = pd.DataFrame(columns=['image_file', 'class', 'x_center', 'y_center', 'width', 'height'])

df['image_file'] = image_list

i=0
for file in df['image_file']:
    label = open(path + file[:-5] +'.txt', 'r').readline().split()
    df.loc[i, 'class'] = int(label[0])
    df.loc[i, 'x_center'] = float(label[1])
    df.loc[i, 'y_center'] = float(label[2])
    df.loc[i, 'width'] = float(label[3])
    df.loc[i, 'height'] = float(label[4])
    i +=1
    
df.head()

,image_file,class,x_center,y_center,width,height
0,agri_0_1009.jpeg,1,0.608398,0.498047,0.541016,0.53125
1,agri_0_1017.jpeg,1,0.496094,0.384766,0.960938,0.753906
2,agri_0_1018.jpeg,1,0.445312,0.483398,0.808594,0.634766
3,agri_0_1020.jpeg,1,0.494141,0.350586,0.730469,0.611328
4,agri_0_1024.jpeg,1,0.601562,0.448242,0.570312,0.595703


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1300 entries, 0 to 1299
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   image_file  1300 non-null   object
 1   class       1300 non-null   object
 2   x_center    1300 non-null   object
 3   y_center    1300 non-null   object
 4   width       1300 non-null   object
 5   height      1300 non-null   object
dtypes: object(6)
memory usage: 61.1+ KB


In [8]:
# creating dataframe in Pascal VOC format with bounding boxes scaled (from originl data)
df_pas_voc = pd.DataFrame(columns = ['filename', 'width', 'height', 'x_min', 'y_min', 'x_max', 'y_max', 'class'])

df_pas_voc['filename'] = df['image_file']
df_pas_voc['class'] = df['class']
df_pas_voc['width'] = 512
df_pas_voc['height'] = 512

for i in range(len(df_pas_voc)):
    df_pas_voc.loc[i, 'x_min'] = df.loc[i, 'x_center'] - (df.loc[i, 'width']/2)
    df_pas_voc.loc[i, 'y_min'] = df.loc[i, 'y_center'] - (df.loc[i, 'height']/2)
    df_pas_voc.loc[i, 'x_max'] = df.loc[i, 'x_center'] + (df.loc[i, 'width']/2)
    df_pas_voc.loc[i, 'y_max'] = df.loc[i, 'y_center'] + (df.loc[i, 'height']/2)

df_pas_voc.head()

,filename,width,height,x_min,y_min,x_max,y_max,class
0,agri_0_1009.jpeg,512,512,0.33789,0.232422,0.878906,0.763672,1
1,agri_0_1017.jpeg,512,512,0.015625,0.007813,0.976563,0.761719,1
2,agri_0_1018.jpeg,512,512,0.041015,0.166015,0.849609,0.800781,1
3,agri_0_1020.jpeg,512,512,0.128906,0.044922,0.859376,0.65625,1
4,agri_0_1024.jpeg,512,512,0.316406,0.15039,0.886718,0.746093,1


In [10]:
# creating dataframe in proper Pascal VOC format with bounding boxes not scaled
df_pascal_voc = pd.DataFrame(columns = ['filename', 'width', 'height', 'x_min', 'y_min', 'x_max', 'y_max', 'class'])

df_pascal_voc['filename'] = df['image_file']
df_pascal_voc['class'] = df['class']
df_pascal_voc['width'] = 512
df_pascal_voc['height'] = 512

for i in range(len(df_pascal_voc)):
    df_pascal_voc.loc[i, 'x_min'] = round((df.loc[i, 'x_center'] - (df.loc[i, 'width']/2))*512)
    df_pascal_voc.loc[i, 'y_min'] = round((df.loc[i, 'y_center'] - (df.loc[i, 'height']/2))*512)
    df_pascal_voc.loc[i, 'x_max'] = round((df.loc[i, 'x_center'] + (df.loc[i, 'width']/2))*512)
    df_pascal_voc.loc[i, 'y_max'] = round((df.loc[i, 'y_center'] + (df.loc[i, 'height']/2))*512)

df_pascal_voc.head()

,filename,width,height,x_min,y_min,x_max,y_max,class
0,agri_0_1009.jpeg,512,512,173,119,450,391,1
1,agri_0_1017.jpeg,512,512,8,4,500,390,1
2,agri_0_1018.jpeg,512,512,21,85,435,410,1
3,agri_0_1020.jpeg,512,512,66,23,440,336,1
4,agri_0_1024.jpeg,512,512,162,77,454,382,1


### Formating data for modeling

In [16]:
# loading images and preprocessing into and array of pixel values
data = []
for filename in df_pas_voc['filename']:
    image = load_img(path + filename, target_size=(224, 224))
    image = img_to_array(image)
    data.append(image)

In [25]:
# converting data (images) into numpy array and scaling pixel intensities
data = np.array(data, dtype='float32') / 255.0
data[0:5]

array([[[[3.6293734e-03, 3.6447521e-03, 3.3986929e-03],
         [3.5678586e-03, 3.5371012e-03, 3.3371781e-03],
         [3.8292964e-03, 3.7677817e-03, 3.5986160e-03],
         ...,
         [1.7993080e-03, 1.6147635e-03, 1.6762784e-03],
         [1.8762015e-03, 1.6455210e-03, 1.7531719e-03],
         [1.8454441e-03, 1.5993848e-03, 1.7531719e-03]],

        [[3.4909651e-03, 3.5063438e-03, 3.2602844e-03],
         [3.4909651e-03, 3.4602077e-03, 3.2602844e-03],
         [3.7831604e-03, 3.7216456e-03, 3.5524799e-03],
         ...,
         [1.8454441e-03, 1.6608997e-03, 1.7224145e-03],
         [1.8146867e-03, 1.6147635e-03, 1.7224145e-03],
         [1.8608228e-03, 1.6147635e-03, 1.7685506e-03]],

        [[3.6139947e-03, 3.6293734e-03, 3.4140716e-03],
         [3.6293734e-03, 3.5986160e-03, 3.3986929e-03],
         [3.7062669e-03, 3.6447521e-03, 3.4755864e-03],
         ...,
         [2.3836987e-03, 2.2145330e-03, 2.2760478e-03],
         [2.4298348e-03, 2.2606691e-03, 2.3529413e-03],
  

In [26]:
type(data)

numpy.ndarray

In [46]:
# creating list of labels
labels = []
for i in range(len(df_pas_voc)):
    labels.append(df_pas_voc.loc[i, 'class']) 
labels[0:5]

[1, 1, 1, 1, 1]

In [41]:
# converting labels into numpy array
labels = np.array(labels)
labels

array([1, 1, 1, ..., 1, 1, 1])

In [42]:
type(labels)

numpy.ndarray

In [49]:
# since there are only twwo unqie labels we need to use tf's utility function
labels = to_categorical(labels)

In [50]:
# creating list of bounding box values
bboxes = []
for i in range(len(df_pas_voc)):
    bboxes.append((df_pas_voc.loc[i, 'x_min'], df_pas_voc.loc[i, 'y_min'], df_pas_voc.loc[i, 'x_max'], df_pas_voc.loc[i, 'y_max']))
    
bboxes[:5]    

[(0.33788999999999997, 0.23242200000000002, 0.878906, 0.763672),
 (0.015625, 0.007813000000000014, 0.976563, 0.761719),
 (0.04101499999999997, 0.16601499999999997, 0.8496090000000001, 0.800781),
 (0.12890649999999998, 0.04492200000000002, 0.8593755000000001, 0.65625),
 (0.316406, 0.15039049999999998, 0.8867180000000001, 0.7460935)]

In [51]:
# converting bounding boxes into numpy array
bboxes = np.array(bboxes, dtype="float32")
bboxes[0:5]

array([[0.33789  , 0.232422 , 0.878906 , 0.763672 ],
       [0.015625 , 0.007813 , 0.976563 , 0.761719 ],
       [0.041015 , 0.166015 , 0.849609 , 0.800781 ],
       [0.1289065, 0.044922 , 0.8593755, 0.65625  ],
       [0.316406 , 0.1503905, 0.886718 , 0.7460935]], dtype=float32)

In [52]:
# creating list of image paths
image_paths = []
for i in range(len(df_pas_voc)):
    image_paths.append((path + df_pas_voc.loc[i, "filename"]))

image_paths[0:5]

['../Data/data/agri_0_1009.jpeg',
 '../Data/data/agri_0_1017.jpeg',
 '../Data/data/agri_0_1018.jpeg',
 '../Data/data/agri_0_1020.jpeg',
 '../Data/data/agri_0_1024.jpeg']

In [53]:
# converting image paths into numpy array
image_paths = np.array(image_paths)